<a href="https://colab.research.google.com/github/ambideXtrous9/T5-FineTuned-Model-for-NewsQA/blob/main/Context_Find_Using_SentenceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = '/content/drive/MyDrive/MTP CODE/NewsQA_SPAN.feather'

In [6]:
df = pd.read_feather(path)
df

,question,answer,ans_pos,paragraph,answer_start,answer_end
0,Who is the managing director of Synergee Capital?,Vikram Dalal,"[133, 145]","""Investors can use a combination of governmen...",133,145
1,What is the yield of 30- and 40-year governmen...,7%,"[565, 567]","""Investors can use a combination of governmen...",565,567
2,What is the name of the ETF 2027 that a conser...,SDL,"[209, 212]","According to financial planners, an example o...",209,212
3,When would a conservative fixed income investo...,2027,"[217, 221]","According to financial planners, an example o...",217,221
4,What year would a conservative fixed income in...,2040,"[260, 264]","According to financial planners, an example o...",260,264
...,...,...,...,...,...,...
481753,When does Uncle Sam reopen for fully vaccinate...,November 8,"[295, 305]",NEW DELHI: This could be the last expansion of...,295,305
481754,When will there be three more weekly flights b...,from second week of November,"[116, 144]",It currently has 23 weekly flights to America....,116,144
481755,What type of 777s would have helped AI have mo...,Boeing,"[306, 312]",It currently has 23 weekly flights to America....,306,312
481756,What was the first wave of AI nonstops?,second,"[11, 17]","Before the second wave this summer, AI had abo...",11,17


In [7]:
df = df.iloc[:50000]

In [8]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [38]:
model.max_seq_length

384

In [39]:
corpus_embeddings = model.encode(df['question'],show_progress_bar=True,convert_to_tensor=True)


Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

In [40]:
idx = 0
question = df.iloc[idx]['question']
question

'Who is the managing director of Synergee Capital?'

In [41]:
question_embedding = model.encode(question,convert_to_tensor=True)

In [51]:
result = util.semantic_search(question_embedding,corpus_embeddings)[0]
result

[{'corpus_id': 47698, 'score': 1.0000001192092896},
 {'corpus_id': 47700, 'score': 1.0000001192092896},
 {'corpus_id': 0, 'score': 1.0},
 {'corpus_id': 9505, 'score': 0.9756023287773132},
 {'corpus_id': 9509, 'score': 0.9756023287773132},
 {'corpus_id': 38240, 'score': 0.8094788789749146},
 {'corpus_id': 38241, 'score': 0.8094788789749146},
 {'corpus_id': 34395, 'score': 0.7935486435890198},
 {'corpus_id': 34402, 'score': 0.793548583984375},
 {'corpus_id': 49378, 'score': 0.785905659198761}]

In [44]:
for item in result:
    print(round(item['score'],2),df.iloc[item['corpus_id']]['paragraph'])

1.0  Bond dealers said yields can spike in the range of 25-100 basis points. A basis point is 0.01 percentage point. Depending on the quantum of selling, yields on bank perpetual bonds could surge 50-100 basis points, said Vikram Dalal, managing director, Synergee Capital. The spike will be even higher in the case of NBFC perpetual bonds, he said. “Smart investors could well sneak in, tapping arbitrage opportunities,” Manglunia said.
1.0  Bond dealers said yields can spike in the range of 25-100 basis points. A basis point is 0.01 percentage point. Depending on the quantum of selling, yields on bank perpetual bonds could surge 50-100 basis points, said Vikram Dalal, managing director, Synergee Capital. The spike will be even higher in the case of NBFC perpetual bonds, he said. “Smart investors could well sneak in, tapping arbitrage opportunities,” Manglunia said.
1.0  "Investors can use a combination of government bonds and low expense passive debt funds to build a ladder for their por

# **Accuaracy**

In [46]:
def predict_context(question):
    question_embedding = model.encode(question,convert_to_tensor=True)
    result = util.semantic_search(question_embedding,corpus_embeddings)[0][0]
    top_context = df.iloc[result['corpus_id']]['paragraph']
    return top_context

In [47]:
def pred(idx):
    ques = df.iloc[idx]['question']
    print("Q : ", ques)
    predicted_context = predict_context(ques)
    print("Actual Context : ", df.iloc[idx]['paragraph'])
    print("Predicted Context : ", predicted_context)

In [48]:
predict = df['question'].apply(predict_context)

In [49]:
res = (predict == df['paragraph']).value_counts()
res

True     49011
False      989
dtype: int64

In [50]:
accuracy = (res[True] / len(predict))*100
accuracy

98.02199999999999